In [ ]:
from aux_fun import get_tokens_inst_iter_folders
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import guitarpro as pygp
import glob

path_to_general_read_folder = "..\..\data\DadaGP\DadaGP-v1.1"
path_to_general_write_folder = "..\..\data\BGTG\BGTG-v0.1"

#### Assigning the rythmic guitar to each track

In [ ]:
# # now let's take a look at the instruments that generally are rythmic
# df_rg_augmented.columns
# # We remove the features columns of the dataset

# df_rg_aug_essentials = df_rg_augmented[['Fichier', 'Track_Name', 'Partie', 'Instrument', 'DadaGP_name', 'Mesure', 'rg-estimation']]

df_rg_aug_essentials = total_df_track[['Fichier', 'Track_Name', 'Partie', 'Instrument', 'Mesure', 'DadaGP_name', 'File_Name', 'rg-estimation']]
display(df_rg_aug_essentials.head())
df_rg_aug_essentials.shape, df_rg_aug_essentials['DadaGP_name'].nunique()

,Fichier,Track_Name,Partie,Instrument,Mesure,DadaGP_name,File_Name,rg-estimation
255,../data/DadaGP8-gpif/12 Stones - Crash-Bass.gpif,Crash\n12 Stones,Bass,bass,9,bass,12 Stones - Crash,0.953768
256,../data/DadaGP8-gpif/12 Stones - Crash-Bass.gpif,Crash\n12 Stones,Bass,bass,10,bass,12 Stones - Crash,0.964169
257,../data/DadaGP8-gpif/12 Stones - Crash-Bass.gpif,Crash\n12 Stones,Bass,bass,11,bass,12 Stones - Crash,0.962112
258,../data/DadaGP8-gpif/12 Stones - Crash-Bass.gpif,Crash\n12 Stones,Bass,bass,12,bass,12 Stones - Crash,0.986466
259,../data/DadaGP8-gpif/12 Stones - Crash-Bass.gpif,Crash\n12 Stones,Bass,bass,13,bass,12 Stones - Crash,0.983138


((298881, 8), 10)

In [ ]:
# Now let's loop again on the tracks and take a look at what instruments are generally rythmic

# First describe column rg-estimation
df_rg_aug_essentials['rg-estimation'].describe()

# We could loop on tracks, within tracks loop on measures and for each measure take the instrument that has the highest rg-estimation
# But that would be very long?
# Maybe we can just say if rg-estimation < thr then the instrument is rythmic

thr = 0.5 # General decision threshold in the paper that is used to determine if an instrument is rythmic or not
# When rg-estimation is near 1, the instrument is lead, when it is near 0 the instrument is rythmic.
df_rg_aug_essentials.loc[:, 'is_rythmic'] = df_rg_aug_essentials['rg-estimation'] < thr

display(df_rg_aug_essentials['is_rythmic'].value_counts())


C:\Users\oanou\AppData\Local\Temp\ipykernel_31820\2434559761.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rg_aug_essentials.loc[:, 'is_rythmic'] = df_rg_aug_essentials['rg-estimation'] < thr


is_rythmic
True     254041
False     44840
Name: count, dtype: int64

In [ ]:
# However, now that I need this I need to build a dataframe with for each track and each instrument its proportion of rythmic measures

total_df_rythmic = pd.DataFrame()
i=0
# For each track we add the DadaGP_name column
for track in tqdm(df_rg_aug_essentials['File_Name'].unique()):
    i=i+1
    df_rythmic = df_rg_aug_essentials[df_rg_aug_essentials['File_Name'] == track].copy() # We copy to avoid SettingWithCopyWarning
    max_nb_measures = df_rythmic['Mesure'].max()
    
    for instrument in df_rythmic['DadaGP_name'].unique():
        df_inst = df_rythmic[df_rythmic['DadaGP_name'] == instrument]
        
        # Proportion of rythmic measures for the instrument (in the whole track and not only in the measures where the instrument plays)
        prop_rythmic = df_inst['is_rythmic'].sum() / max_nb_measures
        if prop_rythmic > 1:
            print(track, max_nb_measures, prop_rythmic)
        df_rythmic.loc[df_rythmic['DadaGP_name'] == instrument, 'prop_rythmic'] = prop_rythmic  
    
    # The instrument with the highest proportion of rythmic measures is the rythmic instrument
    if df_rythmic['prop_rythmic'].isnull().all(): # No rythmic instrument at all in the track
        df_rythmic.loc[:, 'is_track_rythmic'] = False
    
    else:
        # Add column is_track_rythmic set to True at the rows where prop_rythmic is its max
        df_rythmic.loc[:, 'is_track_rythmic'] = df_rythmic['prop_rythmic'] == df_rythmic['prop_rythmic'].max()
          
    # Concatenate the dataframe
    total_df_rythmic = pd.concat([total_df_rythmic, df_rythmic])
    
    # if i==100: # Test on 100 tracks
    #     break

# Around an hour of computation according to tqdm...

  2%|▏         | 24/1315 [00:00<00:17, 73.58it/s]

3 Doors Down - Be Like That 91 2.5824175824175826


  4%|▍         | 56/1315 [00:00<00:18, 69.88it/s]

311 - Homebrew (2) 73 4.780821917808219
311 - Homebrew (2) 73 1.9452054794520548
311 - Homebrew (2) 73 1.8356164383561644


  6%|▌         | 78/1315 [00:01<00:17, 69.95it/s]

...

 95%|█████████▍| 1244/1315 [00:24<00:01, 38.36it/s]

Axel Rudi Pell, Axel - Fool Fool 170 1.8705882352941177
Axel Rudi Pell, Axel - Fool Fool 170 1.5176470588235293
Axel Rudi Pell, Axel - Fool Fool 170 1.5176470588235293


 97%|█████████▋| 1276/1315 [00:25<00:01, 37.97it/s]

Lavigne, Avril - Things I'll Never Say 91 1.4725274725274726


100%|██████████| 1315/1315 [00:26<00:00, 49.17it/s]


In [ ]:
total_df_rythmic_track_level = total_df_rythmic.copy()
total_df_rythmic_track_level = total_df_rythmic_track_level.drop_duplicates(subset=['Track_Name', 'File_Name'])
total_df_rythmic_track_level.head(20)

,Fichier,Track_Name,Partie,Instrument,Mesure,DadaGP_name,File_Name,rg-estimation,is_rythmic,prop_rythmic,is_track_rythmic
255,../data/DadaGP8-gpif/12 Stones - Crash-Bass.gpif,Crash\n12 Stones,Bass,bass,9,bass,12 Stones - Crash,9.537677e-01,False,0.674074,False
634,../data/DadaGP8-gpif/1349 - Deathmarch-Guitar....,Deathmarch\n1349,Guitar,distorted,1,distorted0,1349 - Deathmarch,1.147128e-06,True,1.000000,True
742,../data/DadaGP8-gpif/1349 - Riders Of The Apoc...,Riders Of The Apocalypse\n1349,Bass,bass,3,bass,1349 - Riders Of The Apocalypse,1.927962e-11,True,0.973822,False
1115,"../data/DadaGP8-gpif/1974 AD, Pahilo Junema - ...",Jungi Nishan\nSajal Pradhan,Bass,bass,2,bass,"1974 AD, Pahilo Junema - Jungi Nishan",2.114279e-08,True,0.936170,False
1389,"../data/DadaGP8-gpif/1974 ad, pahilo junema - ...",Manta Mero Nepali\n1974 AD,Track 1,distorted,1,distorted0,"1974 ad, pahilo junema - Manta Mero Nepali",2.982967e-11,True,0.750000,True
1205,"../data/DadaGP8-gpif/1974 AD, Pahilo Junema - ...",Sisire jhari\nAdrian pradhan,Track 1,clean,1,clean0,"1974 AD, Pahilo Junema - Sisire Jhai",2.234772e-07,True,1.000000,True
1445,../data/DadaGP8-gpif/1Kempfi1 - 4 Seasons- 4....,4 Seasons\nKempfi,4,clean,1,clean0,1Kempfi1 - 4 Seasons,3.403829e-07,True,1.000000,True
1761,../data/DadaGP8-gpif/1Kempfi1 - Short & Stupid...,Short & Stupid\nMCA,Spur 1,distorted,1,distorted0,1Kempfi1 - Short & Stupid,5.604797e-05,True,0.840000,False
1895,../data/DadaGP8-gpif/1Kempfi1 - Sunrise-Spur 1...,Sunrise\nKempfi,Spur 1,bass,1,bass,1Kempfi1 - Sunrise,1.273318e-12,True,0.989796,True
2112,../data/DadaGP8-gpif/1st Man For Her - A Peach...,A Peach Fantasy\nBernard Gervais,Bass,bass,19,bass,1st Man For Her - A Peach Fantasy,4.222868e-05,True,0.527273,True
